In [83]:
# https://www.tensorflow.org/tutorials/keras/regression


from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import math
import os
import os.path
from os import path
import scipy
import pickle
import tensorflow as tf
import xgboost as xgb

import sys



from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_lattice as tfl
print(tf.__version__)
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
# import tensorflow_docs as tfdocs
# import tensorflow_docs.plots
# import tensorflow_docs.modeling

# from ttictoc import TicToc



2.1.0


In [2]:
# Import helpers
!#pip install import_ipynb
import import_ipynb
from common import buildDataset
from common import filterBad
from common import splitDataset
from common import normDataset
from common import evaluatePerf
from common import predictPoly
from common import randomImageIndices
from common import evaluateCustom
from common import extractNNWeights
from common import rowOffsets

importing Jupyter notebook from common.ipynb


In [3]:
def flatten(l): 
    if (isinstance(l[0], list)): return [item for sublist in l for item in sublist]
    else: return l

# Collect row data bundled by fileId

In [4]:
dataset, columns = buildDataset('processed.feather')
deltas = rowOffsets()

test_idx = randomImageIndices(int(len(dataset)/109), 0.01)

In [146]:
# back calculating stats for individual rows for hypercube
def buildDatasetForLattice(filename = 'processed_floor.feather'):
    import pandas as pd
    column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'first_val', 'last_val', 'delay']
    
    # raw_dataset =  pd.read_feather('../preprocessing/processed.feather')
#     raw_dataset =  pd.read_feather('/local/ssd/home/mattfel/slac/timetool-ML-models/preprocessing/processed.feather')
    raw_dataset =  pd.read_feather('/scratch/mattfel/data-fs/%s' % filename)

    dataset = raw_dataset.copy()
    dataset.tail()
    
    unmodeledColumns = ['fileId', 'volume', 'rising_weight', 'falling_weight']
    dataset = dataset.dropna()
    dataset = dataset.drop(columns=unmodeledColumns)
    return dataset, [item for item in column_names if item not in unmodeledColumns]

stats_dataset, _ = buildDatasetForLattice('processed.feather')
stats_dataset = filterBad(stats_dataset, 54)
stats_train_dataset, stats_test_dataset, stats_train_labels, stats_test_labels = splitDataset(stats_dataset, 0.2)
hcub_columns = ['row', 'rising_idx', 'falling_idx', 'first_val', 'last_val', 'delay']
stats_features = [stats_test_dataset[col].values for col in hcub_columns[:-1]]

Rejected 6174926 points (56.651264%)


In [5]:
test_set = []
for idx in test_idx:
    features = dataset.iloc[idx*109:(idx+1)*109]
    test_set.append(features)

    

# Load models

In [6]:
def garbageGate(ft, prediction):
    if (ft['falling_idx'] - ft['rising_idx'] < 100 or
        ft['falling_idx'] - ft['rising_idx'] > 500 or
        ft['volume'] < 50 or
        (ft['first_val'] > 30 and ft['last_val'] > 30)):
        return float('NaN')
    else:
        return prediction

In [7]:
# polyfit general
def predictPolyfit(pt, rising_model, falling_model):
    scale = 128.0 # divide rising/falling idx by this number so it fits in fpga precision

    tyr = np.sum([m * math.pow(pt['rising_idx'] / scale,i) for i,m in enumerate(rising_model[::-1])])
    tyf = np.sum([m * math.pow(pt['falling_idx'] / scale,i) for i,m in enumerate(falling_model[::-1])])
    
    if (pt['first_val'] >= 30): return tyf # only use falling
    elif (pt['last_val'] >= 30): return tyr # only use rising
    return garbageGate(pt,(tyr + tyf) / 2) # use both rising and falling

def polyFit2(ft):
    rising_model = [54.93240949884357,-1196.8992955773579,1880.8014648783726]
    falling_model = [42.17895475957633,-1117.8885336758663,3174.7423125408845]
    return predictPolyfit(ft, rising_model, falling_model)

def polyFit3(ft):
    rising_model = [-3.453292877552903,82.28053267025867,-1257.1970620164395,1914.700283489191]
    falling_model = [-2.4793743678901543,73.19222899717427,-1235.8065880531187,3308.967740573062]
    return predictPolyfit(ft, rising_model, falling_model)
    
def polyFit4(ft):
    rising_model = [0.2983872186312137,-6.794932480985469,94.59195292870838,-1274.0845848248348,1921.4176948170698]
    falling_model = [0.17045218683904267,-5.383933637827994,90.48238200637373,-1277.9836931678628,3344.180658908667]
    return predictPolyfit(ft, rising_model, falling_model)


In [16]:
# FCNN general

def normalize(ft):
    with open('../networks/train_stats', 'rb') as f:
        stats = pickle.load(f)
    todrop = (['delay'] if 'delay' in ft else []) + (['fileId'] if 'fileId' in ft else [])
    normed = ft.copy()
    normed['volume'] = ft['volume'] / 4096
    normed = (normed.drop(columns=todrop) - stats['mean']) / stats['std']
    
    col_order = ['row',  'rising_idx',  'falling_idx',       'volume',  'rising_weight',  'falling_weight',  'first_val',  'last_val']
    ordered = normed.reindex(columns=col_order)
    return ordered
    
def buildNN(hLayers, actFuncs):
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    nn = []
    input_dim = 8
    for i in range(len(hLayers)):
        if (i == 0):
            shape = [input_dim]
        else:
            shape = [hLayers[i-1]]
        nn.append(layers.Dense(hLayers[i], activation=actFuncs[i], input_shape = shape))
    nn.append(layers.Dense(1))
    model = keras.Sequential(nn)

    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
    return model

def predictNN(image, model):
    nimage = normalize(image)
    raw_pred = model.predict(nimage).flatten() #(image.to_frame().transpose())[0][0]
    return [garbageGate(image.iloc[i],raw_pred[i]) for i in range(len(image))]

def predictNN_64_RELU(image, model):
    if (model == None):
        hDims = [64]
        actFuncs = ['relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_64_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_64_Leaky(image, model):
    if (model == None):
        hDims = [64]
        
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_64_Leaky_strong') #load
    return predictNN(image, model), model

def predictNN_32_32_RELU(image, model):
    if (model == None):
        hDims = [32, 32]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_32_32_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_32_32_Leaky(image, model):
    if (model == None):
        hDims = [32, 32]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_32_32_Leaky_strong') #load
    return predictNN(image, model), model
    

def predictNN_16_16_RELU(image, model):
    if (model == None):
        hDims = [16, 16]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_16_16_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_16_16_Leaky(image, model):
    if (model == None):
        hDims = [16, 16]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_16_16_Leaky_strong') #load
    return predictNN(image, model), model



def predictNN_8_8_RELU(image, model):
    if (model == None):
        hDims = [8, 8]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_8_8_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_8_8_Leaky(image, model):
    if (model == None):
        hDims = [8, 8]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_8_8_Leaky_strong') #load
    return predictNN(image, model), model


def predictNN_4_4_RELU(image, model):
    if (model == None):
        hDims = [4, 4]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_4_4_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_4_4_Leaky(image, model):
    if (model == None):
        hDims = [4, 4]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_4_4_Leaky_strong') #load
    return predictNN(image, model), model

def predictNN_16_4_RELU(image, model):
    if (model == None):
        hDims = [16, 4]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_16_4_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_16_4_Leaky(image, model):
    if (model == None):
        hDims = [16, 4]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_16_4_Leaky_strong') #load
    return predictNN(image, model), model
    
def predictNN_8_4_RELU(image, model):
    if (model == None):
        hDims = [8, 4]
        actFuncs = ['relu', 'relu']
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_8_4_RELU_strong') #load
    return predictNN(image, model), model

def predictNN_8_4_Leaky(image, model):
    if (model == None):
        hDims = [8, 4]
        actFuncs = [lambda x : tf.nn.leaky_relu(x, 0.125), lambda x : tf.nn.leaky_relu(x, 0.125)]
        model = buildNN(hDims, actFuncs)
        model.load_weights('../networks/NN_8_4_Leaky_strong') #load
    return predictNN(image, model), model

In [59]:
def buildXG(alpha, colsampbytree, base_score, lr, lmbda, num_trees, max_depth):
    return xgb.XGBRegressor(objective ='reg:linear', base_score = base_score, 
                              colsample_bytree = colsampbytree, learning_rate = lr,
                              max_depth = max_depth, alpha = alpha, n_estimators = num_trees, reg_lambda = lmbda)

def predictXG(image, model):
#     nimage = normalize(image)
    nimage = image.drop(columns=['fileId', 'delay'])
    nimage['volume'] = nimage['volume'] / 4096
    raw_pred = model.predict(nimage).flatten() #(image.to_frame().transpose())[0][0]
    return [garbageGate(image.iloc[i],raw_pred[i]) for i in range(len(image))]

def predictXGBoost_10_5(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 10, 5)
        model.load_model('../networks/XGBoost_10_5_strong')
    return predictXG(image,model), model
def predictXGBoost_20_5(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 20, 5)
        model.load_model('../networks/XGBoost_20_5_strong')
    return predictXG(image,model), model
def predictXGBoost_25_5(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 25, 5)
        model.load_model('../networks/XGBoost_25_5_strong')
    return predictXG(image,model), model
def predictXGBoost_10_6(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 10, 6)
        model.load_model('../networks/XGBoost_10_6_strong')
    return predictXG(image,model), model
def predictXGBoost_20_6(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 20, 6)
        model.load_model('../networks/XGBoost_20_6_strong')
    return predictXG(image,model), model
def predictXGBoost_40_4(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 40, 4)
        model.load_model('../networks/XGBoost_40_4_strong')
    return predictXG(image,model), model
def predictXGBoost_64_4(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 64, 4)
        model.load_model('../networks/XGBoost_64_4_strong')
    return predictXG(image,model), model
def predictXGBoost_96_4(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 96, 4)
        model.load_model('../networks/XGBoost_96_4_strong')
    return predictXG(image,model), model
def predictXGBoost_128_4(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 128, 4)
        model.load_model('../networks/XGBoost_128_4_strong')
    return predictXG(image,model), model
def predictXGBoost_128_3(image, model):
    if (model == None):
        model = buildXG(10, 0.3, 0.5, 0.9, 2, 128, 3)
        model.load_model('../networks/XGBoost_128_3_strong')
    return predictXG(image,model), model


In [156]:

def predictHypercube(image, model):
#     nimage = normalize(image)
    columns = ['row', 'rising_idx', 'falling_idx', 'first_val', 'last_val', 'delay']
    nimage = [image[col].values for col in columns[:-1]]
    raw_pred = model.predict(nimage).flatten() 
    return [garbageGate(image.iloc[i],raw_pred[i]) for i in range(len(image))]



def predictHypercube_16_2(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_16_2.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model

def predictHypercube_4_2(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_4_2.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model

def predictHypercube_4_3(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_4_3.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model

def predictHypercube_4_4(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_4_4.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model
    
def predictHypercube_4_5(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_4_5.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model

def predictHypercube_8_2(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_8_2.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model
    
def predictHypercube_8_3(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_8_3.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model

def predictHypercube_8_4(image, model):
    if (model == None):
        model = keras.models.load_model('../networks/hypercube_sweep_8_4.h5', custom_objects={
            "CategoricalCalibration": tfl.layers.CategoricalCalibration,
            "PWLCalibration": tfl.layers.PWLCalibration,
            "Lattice": tfl.layers.Lattice,
            "Linear": tfl.layers.Linear
        })
    return predictHypercube(image,model), model



# Kernel switches

In [157]:
def predictImage(model_name, image, deltas, row_thresh, model):
    if (model_name == 'polyfit2'):
        return [polyFit2(image.iloc[i]) for i in range(row_thresh)], None
    elif (model_name == 'polyfit3'):
        return [polyFit3(image.iloc[i]) for i in range(row_thresh)], None
    elif (model_name == 'polyfit4'):
        return [polyFit4(image.iloc[i]) for i in range(row_thresh)], None
    elif (model_name == 'NN_64_RELU_strong'):
        preds, model = predictNN_64_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_64_Leaky_strong'):
        preds, model = predictNN_64_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_32_32_RELU_strong'):
        preds, model = predictNN_32_32_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_32_32_Leaky_strong'):
        preds, model = predictNN_32_32_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_16_16_RELU_strong'):
        preds, model = predictNN_16_16_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_16_16_Leaky_strong'):
        preds, model = predictNN_16_16_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_8_8_RELU_strong'):
        preds, model = predictNN_8_8_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_8_8_Leaky_strong'):
        preds, model = predictNN_8_8_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_4_4_RELU_strong'):
        preds, model = predictNN_4_4_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_4_4_Leaky_strong'):
        preds, model = predictNN_4_4_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_16_4_RELU_strong'):
        preds, model = predictNN_16_4_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_16_4_Leaky_strong'):
        preds, model = predictNN_16_4_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_8_4_RELU_strong'):
        preds, model = predictNN_8_4_RELU(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'NN_8_4_Leaky_strong'):
        preds, model = predictNN_8_4_Leaky(image, model)
        return preds[:row_thresh], model
    elif (model_name == 'XGBoost_10_5_strong'):
            preds, model = predictXGBoost_10_5(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_20_5_strong'):
            preds, model = predictXGBoost_20_5(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_25_5_strong'):
            preds, model = predictXGBoost_25_5(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_10_6_strong'):
            preds, model = predictXGBoost_10_6(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_20_6_strong'):
            preds, model = predictXGBoost_20_6(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_40_4_strong'):
            preds, model = predictXGBoost_40_4(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_64_4_strong'):
            preds, model = predictXGBoost_64_4(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_96_4_strong'):
            preds, model = predictXGBoost_96_4(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_128_4_strong'):
            preds, model = predictXGBoost_128_4(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'XGBoost_128_3_strong'):
            preds, model = predictXGBoost_128_3(image, model)
            return preds[:row_thresh], model
    elif (model_name == 'Hypercube_16_2'):
        preds, model = predictHypercube_16_2(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_4_2'):
        preds, model = predictHypercube_4_2(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_4_3'):
        preds, model = predictHypercube_4_3(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_4_4'):
        preds, model = predictHypercube_4_4(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_4_5'):
        preds, model = predictHypercube_4_5(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_8_2'):
        preds, model = predictHypercube_8_2(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_8_3'):
        preds, model = predictHypercube_8_3(image,model)
        return preds[:row_thresh], model
    elif (model_name == 'Hypercube_8_4'):
        preds, model = predictHypercube_8_4(image,model)
        return preds[:row_thresh], model

        

In [158]:
models_list = [#'polyfit2', 'polyfit3', 'polyfit4',
#                'NN_64_RELU_strong', 
               #'NN_64_Leaky_strong', 'NN_32_32_RELU_strong', 'NN_32_32_Leaky_strong']
#                 'NN_16_16_RELU_strong','NN_16_16_Leaky_strong','NN_8_8_RELU_strong','NN_8_8_Leaky_strong','NN_4_4_RELU_strong','NN_4_4_Leaky_strong','NN_16_4_RELU_strong','NN_16_4_Leaky_strong','NN_8_4_RELU_strong','NN_8_4_Leaky_strong',
#                 'XGBoost_10_5_strong','XGBoost_20_5_strong','XGBoost_25_5_strong','XGBoost_10_6_strong','XGBoost_20_6_strong','XGBoost_40_4_strong','XGBoost_64_4_strong','XGBoost_96_4_strong','XGBoost_128_4_strong','XGBoost_128_3_strong',
    #            'Hypercube_16_2', 
    'Hypercube_4_2', 'Hypercube_4_3', 'Hypercube_4_4', 'Hypercube_4_5', 'Hypercube_8_2', 'Hypercube_8_3', 'Hypercube_8_4', 
              ]
    
    

In [159]:
for model_name in models_list:
    model = None
    
    row_thresh = 54
    preds = []
    img_labels = []
    line_labels = []
    stats = []

    i = 0
    for pt in test_set:
        i = i + 1
#         if (i % 500 == 0): print(i)
        # Get top level label for row
        img_label = pt.iloc[0]['delay']
        img_labels.append(img_label)
        # Get prediction per row on test point
        image_pred, model = predictImage(model_name, pt, deltas, row_thresh, model) 
        line_labels.append(pt.iloc[:]['delay'].tolist()[:row_thresh])
        preds.append(image_pred)
#         print('got: %.2f, want: %.2f' % (np.nanmean([a+b for a,b in zip(image_pred,deltas)]), img_label))
#         print([a+b for a,b in zip(image_pred,deltas)][0:5])
#         print([a+b for a,b in zip(pt.iloc[0:5]['delay'], deltas)])
#         print()
        
        stats.append({'pred_mean': np.nanmean(image_pred), 
                      'pred_var': np.nanvar(image_pred), 
                      'avg_err': abs(np.nanmean([a + b for a,b in zip(image_pred, deltas)] - img_label))})

    mae = np.nansum([x['avg_err'] for x in stats]) / len(stats)
    mse = np.nansum([x['avg_err']**2 for x in stats]) / len(stats)
    av = np.nansum([x['pred_var'] for x in stats]) / len(stats)
    print('%s: mse,mae,avg_var: %.2f,%.2f,%.2f' % (model_name,mse, mae,av))
    
    model_preds = model.predict(stats_features)
    evaluatePerf(stats_test_labels.tolist(), model_preds)

Hypercube_4_2: mse,mae,avg_var: 485.10,10.59,438159.79
MSE, MAE, r2: 1706.696126,18.442829,0.998399
Hypercube_4_3: mse,mae,avg_var: 451.31,10.13,439160.12
MSE, MAE, r2: 1763.654033,19.027192,0.998345
Hypercube_4_4: mse,mae,avg_var: 9221.38,75.34,416353.43
MSE, MAE, r2: 12027.770572,76.253949,0.988716
Hypercube_4_5: mse,mae,avg_var: 5961.15,58.36,415062.92
MSE, MAE, r2: 11295.924449,75.016519,0.989402
Hypercube_8_2: mse,mae,avg_var: 747.92,14.73,437526.86
MSE, MAE, r2: 2346.181674,21.620354,0.997799
Hypercube_8_3: mse,mae,avg_var: 13989.99,94.77,428276.17
MSE, MAE, r2: 18364.189895,106.183202,0.982771
Hypercube_8_4: mse,mae,avg_var: 13459.43,90.35,419870.68
MSE, MAE, r2: 18256.849142,102.332042,0.982872


944993
944993
MSE, MAE, r2: 2933.534350,21.632795,0.997248


In [ ]:
print(pt.iloc[0].tolist())

In [20]:
# print(evaluatePerf(flatten(line_labels)[0:1*54], flatten(preds)[0:1*54]))
# print([a+b for a,b in zip(flatten(line_labels)[0:54], deltas[0:54])])
# print([a+b for a,b in zip(flatten(preds)[0:54], deltas[0:54])])
# np.nanmean([a+b for a,b in zip(flatten(preds)[0:54], deltas[0:54])])

# for pt in test_set[0:1]:
#     print('%d=%d' % (pt.iloc[0]['fileId'], pt.iloc[0]['volume']))

# print(test_set[0])


# import pickle
# with open('/scratch/mattfel/testset', 'rb') as f:
#     t = pickle.load(f)
# with open('/scratch/mattfel/testlbl', 'rb') as f:
#     l = pickle.load(f)


# preds = [polyFit2(t.iloc[i]) for i in range(len(t))]



In [38]:
evaluatePerf(l.tolist(),preds)

MSE, MAE, r2: 1931.824895,25.172853,0.998188


In [53]:
for s in stats:
    print(s)

{'pred_mean': 694.1585988354038, 'pred_var': 252710.77694735868, 'avg_err': 5.731842078647074}
{'pred_mean': 1073.5753206041124, 'pred_var': 608382.3964831432, 'avg_err': 196.13332060411255}
{'pred_mean': -1715.0747, 'pred_var': 547173.9, 'avg_err': 1.7079962836372042}
{'pred_mean': 777.8958498636881, 'pred_var': 203391.54544868786, 'avg_err': 6.443122590960836}
{'pred_mean': 984.3070086775155, 'pred_var': 159280.09415394586, 'avg_err': 21.67804316027422}
{'pred_mean': -282.76038000718603, 'pred_var': 543193.4671427888, 'avg_err': 3.2012426343234157}
{'pred_mean': -3.4089186, 'pred_var': 544476.25, 'avg_err': 82.01734807400172}
{'pred_mean': 289.27562657407685, 'pred_var': 465237.8441664559, 'avg_err': 5.349861868194489}
{'pred_mean': 459.11592550838697, 'pred_var': 464753.0475834671, 'avg_err': 114.48316080250467}
{'pred_mean': -772.7942, 'pred_var': 556836.94, 'avg_err': 58.07951352380823}
